## Making Truncation Galaxies ##

* Goal is to make models of truncated galaxies with GALFIT, and then refit them with single Sersic component.  
* For LCS paper 2, we want to see if we take the external sample, truncate, and then fit with a sersic, do we recover the core sample.
* If this works, what does it predict for difference in SFR or sSFR between core and external samples.



In [4]:
import numpy as np
from matplotlib import pyplot as plt
from astropy.io import fits

## Galfit Parameters ##

* from README.pdf
* truncation function is hyperbolic tangent function
* rbreak is the break radius where the profile is 99% of the original, i.e. untruncated, model flux at that radius.
* $\Delta r_{soft}$ is the softening length, so that $r = r_{break} \pm \Delta r_{soft}$ is where the flux drops to 1% that of an un-truncated model at the same radius (the $\pm$ sign depends on whether the truncation is inner or outer)


### Implementing Truncation in Input File ###

From section 8.3.5 of README.pdf

* Item To – Outer truncation by component num- ber N

## Example Input File ##

It took me a while to figure out the right format to get truncation to work, so I'm recording an example input file here.

```
# IMAGE PARAMETERS
A) MKW11-00-model.fits              # Input data image (FITS file)
B) galfit-testout-trunc1.fits       # Name for the output image
C) MKW11-00-galfit-cutout-unc-poisson24.fits                # Sigma image name (made from data if blank or "none") 
D) MKW11psf_starv2.fits     # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF oversampling factor relative to data
F) MKW11-00-galfit-mask24.fits           # Pixel mask (ASCII file or FITS file with non-0 values)
H) 1 50 1 50     # Image region to fit (xmin xmax ymin ymax)
I) 50 50             # Size of convolution box (x y)
J) 18.53              # Magnitude photometric zeropoint 
K) 2.45000   2.45000         # Plate scale (dx dy)  [arcsec/pix]
O) regular                # Display type (regular, curses, both)
P) 0                   # Create output image only? (1=yes; 0=optimize) 
S) 0                   # Modify/create objects interactively?
 
# Object number: 1 
 0) sersic             # Object type 
 1)     25.0      25.0 1 1  # position x, y        [pixel] 
 3) 11.5      1       # total magnitude     
 4)     6.2       1       #     R_e              [Pixels] 
 5)  2.84       1       # Sersic exponent (deVauc=4, expdisk=1)   
 9)  0.37       0       # axis ratio (b/a)    
10) 99.57       0       # position angle (PA)  [Degrees: Up=0, Left=90] 
 Z) 0                  # Output option (0 = residual, 1 = Don't subtract)  
 To)  2  # outer truncation radius 

# Object number: 2
T0) radial   # truncation type 
T4) 12.  # break radius 
T5)  3  # softening radius 

# Object number: 3 
 0) sky             # Object type 
 1)      0.0   1  # sky background at center of fitting region [ADUs] 
 2) 0      0       # dsky/dx (sky gradient in x)    
 3) 0      0       # dsky/dy (sky gradient in y) 
 Z) 0                  # Output option (0 = residual, 1 = Don't subtract)  


```

This works, but it only creates the model.  

Oddly, when I run a test image with what seems like the same parameters, I get a multi-extension FITS file.

```
# IMAGE PARAMETERS
A) test.fits              # Input data image (FITS file)
B) galfit-output.fits       # Name for the output image
H) 1 50 1 50     # Image region to fit (xmin xmax ymin ymax)
J) 20.00              # Magnitude photometric zeropoint 
K) 1.00000   1.00000         # Plate scale (dx dy)  [arcsec/pix]
O) regular                # Display type (regular, curses, both)
P) 0                   # Create output image only? (1=yes; 0=optimize) 
S) 0                  # Modify/create objects interactively?
 
# Object number: 1 
 0) sersic             # Object type 
 1)     25.0      25.0 1 1  # position x, y        [pixel] 
 3) 10.00      1       # total magnitude     
 4)    10.00       1       #     R_e              [Pixels] 
 5)  2.00       1       # Sersic exponent (deVauc=4, expdisk=1)   
 9)  0.80       1       # axis ratio (b/a)    
10) 30.00       1       # position angle (PA)  [Degrees: Up=0, Left=90] 
 Z) 0                  # Output option (0 = residual, 1 = Don't subtract)  
To) 2    # outer truncation radius 

# Object number: 2 
T0) radial   # truncation type 
T4) 30.00  # break radius 
T5)  0.50  # softening radius 
 
# Object number: 3 
 0) sky             # Object type 
 1)      0.0   1  # sky background at center of fitting region [ADUs] 
 2) 0      0       # dsky/dx (sky gradient in x)    
 3) 0      0       # dsky/dy (sky gradient in y) 
 Z) 0                  # Output option (0 = residual, 1 = Don't subtract)  


```